In [1]:
import pandas as pd

from factue.utils.vars import PROJECT_ROOT
from factue.utils.viz import disp

In [2]:
sample_out = (
    PROJECT_ROOT
    / "data/02_cleaned_reference_azure_openai"
    / "dev"
    / "dev-eng"
    / "batch_0000.parquet"
)
df = pd.read_parquet(sample_out)
disp(df[["text", "reference", "check"]])

,text,reference,check
0,"The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More None",Photo shows Louis Armstrong as a child,"{ ""questions"": [ { ""id"": ""meaning_preserved"", ""answer"": ""false"", ""errors"": [""The claim inaccurately states that the photo shows Louis Armstrong as a child, while the post does not mention him or any photo.""] }, { ""id"": ""correct_named_entities"", ""answer"": ""false"", ""errors"": [""The claim mentions Louis Armstrong, but the post does not reference him by name.""] }, { ""id"": ""correct_numbers"", ""answer"": ""true"", ""errors"": [] }, { ""id"": ""no_added_data"", ""answer"": ""false"", ""errors"": [""The claim adds the context of a photo, which is not present in the original text.""] }, { ""id"": ""no_missing_data"", ""answer"": ""true"", ""errors"": [] }, { ""id"": ""alternative_claim_formulation"", ""answer"": ""false"", ""errors"": [""A better formulation could be: 'The post discusses a 7-year-old boy employed by the Karnofsky family, but does not mention Louis Armstrong or any photo.'""] } ] }"
1,"Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 mill Co This little guy was rescued after his mother was murdered by a hunter. His eyes say it all, don't you think? If you want to ban trophy hunting forever, spread this everywhere. Image Credit: FB. Changes: cropped, resized, text added. https://bit.ly/3b2vwas",This leopard cub's mother was killed by a trophy hunter,"{ ""questions"": [ { ""id"": ""meaning_preserved"", ""answer"": ""true"", ""errors"": [] }, { ""id"": ""correct_named_entities"", ""answer"": ""false"", ""errors"": [""The claim refers to a 'leopard cub', but the original text does not specify the species of the cub.""] }, { ""id"": ""correct_numbers"", ""answer"": ""true"", ""errors"": [] }, { ""id"": ""no_added_data"", ""answer"": ""true"", ""errors"": [] }, { ""id"": ""no_missing_data"", ""answer"": ""true"", ""errors"": [] }, { ""id"": ""alternative_claim_formulation"", ""answer"": ""true"", ""errors"": [] } ] }"
2,Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert Magarmacch // Heavy Rain //Hyderabad //Crocodile // Alert None,Videos show current situation of Hyderabad amid heavy rain,"{ ""questions"": [ { ""id"": ""meaning_preserved"", ""answer"": ""false"", ""errors"": [""The claim does not accurately reflect the content of the post, which does not mention videos or the current situation.""] }, { ""id"": ""correct_named_entities"", ""answer"": ""true"", ""errors"": [] }, { ""id"": ""correct

In [9]:
import pandas as pd
import json
from typing import Any, Dict

import ast


def safe_json_loads(s):
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        return ast.literal_eval(s)


def extract_json_body(s):
    start = s.find("{")
    end = s.rfind("}")
    if start != -1 and end != -1 and start < end:
        return s[start : end + 1]
    else:
        raise ValueError("No valid JSON object found")


def expand_json_column(df, column_name):
    expanded_rows = []
    for idx, row in df.iterrows():
        check = extract_json_body(row[column_name])
        json_data = safe_json_loads(check)
        questions = json_data.get("questions", [])
        record = {}
        for q in questions:
            q_id = q.get("id")
            answer = q.get("answer")
            errors = q.get("errors", [])
            record[f"{q_id}_answer"] = answer
            record[f"{q_id}_errors"] = errors
        expanded_rows.append(record)
    expanded_df = pd.DataFrame(expanded_rows)
    return pd.concat([df.reset_index(drop=True), expanded_df], axis=1)


# Expand the JSON column into new columns
df_expanded = expand_json_column(df[["text", "reference", "check"]], "check")
disp(df_expanded)

,text,reference,check,meaning_preserved_answer,meaning_preserved_errors,correct_named_entities_answer,correct_named_entities_errors,correct_numbers_answer,correct_numbers_errors,no_added_data_answer,no_added_data_errors,no_missing_data_answer,no_missing_data_errors,alternative_claim_formulation_answer,alternative_claim_formulation_errors
0,"The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home. He was originally given homework to get food because he was a starving kid. He remained under the Jewish families employ, until he was 12 Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families. They really admired his musical talent.Later, when he became a professional … See More None",Photo shows Louis Armstrong as a child,"{ ""questions"": [ { ""id"": ""meaning_preserved"", ""answer"": ""false"", ""errors"": [""The claim inaccurately states that the photo shows Louis Armstrong as a child, while the post does not mention him or any photo.""] }, { ""id"": ""correct_named_entities"", ""answer"": ""false"", ""errors"": [""The claim mentions Louis Armstrong, but the post does not reference him by name.""] }, { ""id"": ""correct_numbers"", ""answer"": ""true"", ""errors"": [] }, { ""id"": ""no_added_data"", ""answer"": ""false"", ""errors"": [""The claim adds the context of a photo, which is not present in the original text.""] }, { ""id"": ""no_missing_data"", ""answer"": ""true"", ""errors"": [] }, { ""id"": ""alternative_claim_formulation"", ""answer"": ""false"", ""errors"": [""A better formulation could be: 'The post discusses a 7-year-old boy employed by the Karnofsky family, but does not mention Louis Armstrong or any photo.'""] } ] }",false,"['The claim inaccurately states that the photo shows Louis Armstrong as a child, while the post does not mention him or any photo.']",false,"['The claim mentions Louis Armstrong, but the post does not reference him by name.']",true,[],false,"['The claim adds the context of a photo, which is not present in the original text.']",true,[],false,"[""A better formulation could be: 'The post discusses a 7-year-old boy employed by the Karnofsky family, but does not mention Louis Armstrong or any photo.'""]"
1,"Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 Trophy hunting is horrific. 😥😥😥 mill Co This little guy was rescued after his mother was murdered by a hunter. His eyes say it all, don't you think? If you want to ban trophy hunting forever, spread this everywhere. Image Credit: FB. Changes: cropped, resized, text added. https://bit.ly/3b2vwas",This leopard cub's mother was killed by a trophy hunter,"{ ""questions"": [ { ""id"": ""meaning_preserved"", ""answer"": ""true"", ""errors"": [] }, { ""id"": ""correct_named_entities"", ""answer"": ""false"", ""errors"": [""The claim refers to a 'leopard cub', but the original text does not specify the species of the cub.""] }, { ""id"": ""correct_num